In [2]:
!pip install -q langchain langchain_community langchain-openai openai duckduckgo-search

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 101.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [3]:
!pip install -q ddgs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 56.7 MB/s eta 0:00:00


In [ ]:
from langchain_openai import ChatOpenAI
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_core.tools import tool
from langchain.agents import create_react_agent, AgentExecutor
from langchain import hub # for predifined prompts
from dotenv import load_dotenv
import requests


In [ ]:
load_dotenv()
llm = ChatOpenAI()

In [7]:
# Stem - 1: Create Tool
search_tool = DuckDuckGoSearchRun()

@tool
def get_weather_data(city:str)->str:
  """
  This function fetches the current weather data for a given city.
  """

  url = f"http://api.weatherstack.com/current?access_key=c4b1761e8bf93a8495af4c863f84bcb8&query={city}"

  response = requests.get(url)
  return response.json()


In [8]:
# Step - 2: Pull the ReAct Prompt from langchain hub
prompt = hub.pull("hwchase17/react") # ReAct -> reasoning + action prompt (a design pattern for agent)

In [9]:
# Step - 3: create react agent with pulled prompt
agent = create_react_agent(
    llm=llm,
    tools=[search_tool,get_weather_data],
    prompt=prompt
)

In [10]:
# Step 4: Wrapt it with Agent Executer
agent_executer = AgentExecutor(
    agent = agent,
    tools = [search_tool,get_weather_data],
    verbose = True
)

In [11]:
# Step - 5: Invoke
response = agent_executer.invoke({"input": "Find the capital of Punjab and then find its temperature"})
print(response)

# blue and yellow is obeservation from tool
# green is thought



> Entering new AgentExecutor chain...
I should search for the capital of Punjab first and then use get_weather_data to find its temperature.
Action: duckduckgo_search
Action Input: "capital of Punjab"The Union of India was consequently established from 9 former Indian provinces (East Punjab , United Provinces, Central Provinces, Madras, Bombay, Bihar, Orissa, West Bengal and Assam)... 1 day ago - Punjab (Punjabi, Urdu: پنجاب, pronounced [pə̞ɲˈdʒäːb] ⓘ) is a province of Pakistan. With a population of over 127 million, it is the most populous province in Pakistan and the second most populous subnational polity in the world. Located in the central-eastern region of the country, it has the largest economy, contributing the most to national GDP in Pakistan. Lahore is the capital ... 1 day ago - The Punjab Province, officially the Province of the Punjab , was a province of British India, with its capital in Lahore and summer capitals in Murree and Simla. At its greatest extent, it stretche

In [12]:
response

{'input': 'Find the capital of Punjab and then find its temperature',
 'output': 'The capital of Punjab, India is Chandigarh and its temperature is 31°C.'}

In [ ]:
# prefferable using langGraph for industry grade application due to scalablity